In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import os
import geopandas as gpd
import pandas as pd

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
)
from utils.downloads import _prepare_cache

# População a menos de 1km de parques urbanos e lineares

Pessoas de outras cidades também estão dentro das áreas de 1km de parques, porém optei por não considerá-las ao fazer a contagem.

Eu interpretei esse título como <=1km, não como <1km

In [3]:
data_path = get_data_diretorio()

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

# Dependências

Este notebook depende dos arquivos resultantes dos notebooks '.overlay_setores22_buff_parq', '../../arborizacao_viaria/carregar_dados/malha_distritos' e '.""/malha_parques_lineares_urbanos'

In [5]:
gdf_distritos = gpd.read_parquet(
    os.path.join(
        assets_path,
        'distrito_ibge.parquet'
    )
)

In [6]:
gdf_parq_urb_line=gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_urbanos_e_lineares.parquet'
    )
)

In [7]:
gdf_buffed_setor = gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_lineares_e_urbanos',
        'overlay_setores22_buff_parques_urb_line.parquet'        
    )
)

# sla

In [8]:
gdf_distritos.columns

Index(['CD_MUN', 'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_RGINT', 'NM_RGINT',
       'CD_RGI', 'NM_RGI', 'CD_CONCURB', 'NM_CONCURB', 'AREA_KM2', 'total_pop',
       'total_dom', 'geometry'],
      dtype='object')

In [9]:
for i, distrito in gdf_distritos.iterrows():
     gdf_distritos.loc[gdf_distritos.index==i, 'pop_dist_recort'] =sum(
        gdf_buffed_setor
        .loc[gdf_buffed_setor['CD_DIST']==distrito['CD_DIST'], 'pop_recort']
    )

In [10]:
valores_unicos = gdf_distritos['pop_dist_recort'].unique()
gdf_distritos.loc[~gdf_distritos['pop_dist_recort'].isin(valores_unicos)]

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,pop_dist_recort


In [11]:
gdf_distritos.shape

(96, 15)

# Conferir 